# Imports

In [1]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import DirectoryLoader
from langchain.indexes import SQLRecordManager, index
from pinecone import Pinecone, PodSpec
import os

from dotenv import load_dotenv
load_dotenv()

True

# Loading Data

In [2]:
loader = DirectoryLoader(
    path="./data", 
    glob="*.csv", 
    loader_cls=CSVLoader,
    show_progress=True)

docs = loader.load()

print(docs[20].page_content)

100%|██████████| 2/2 [00:00<00:00, 108.25it/s]

Title: Black Adam
Runtime (minutes): 125
Language: en
Overview: Nearly 5,000 years after he was bestowed with the almighty powers of the Egyptian gods—and imprisoned just as quickly—Black Adam is freed from his earthly tomb, ready to unleash his unique form of justice on the modern world.
Release Date: 2022-10-19
Genre: Action, Adventure, Science Fiction
Keyword: lightning, superhero, anti hero, based on comic, demon, superhero team, duringcreditsstinger, dc extended universe (dceu)
Recommendation: Black Panther: Wakanda Forever, Avatar: The Way of Water, The Woman King, M3GAN, Troll, Thor: Love and Thunder, Glass Onion: A Knives Out Mystery, Terrifier 2, Smile, Avatar, Top Gun: Maverick, The Guardians of the Galaxy Holiday Special, Guillermo del Toro's Pinocchio, Puss in Boots: The Last Wish, Bullet Train, Violent Night, Spider-Man: No Way Home, Ant-Man and the Wasp: Quantumania, Shazam! Fury of the Gods, Jeepers Creepers: Reborn, Fall
Cast: Dwayne Johnson, Bodhi Sabongui, Djimon Houn

# Splitting Data

In [3]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [4]:
print(splits[9].page_content)

Cast: Ratna Pathak Shah, Habib Al Aidroos, Prakash Raj, Jaimini Pathak, Kiran Kumar, Shahnawaz Bhatt, Babrak Akbari, Serena Walia, Ranjit Punia, John Abraham, Jacqueline Fernandez, Elham Ehsas, Rakul Preet Singh, Rajit Kapoor, Mehroos Mir, Mohd Talib, Nimish Desai
Crew: Kuldeep Yadav, Akshay Jayantilal Gada, Nakul Chugh, Soumik Mukherjee, Yogendra Mogre, Amritpal Singh, Minnakshi Das, Surbhi Yadav, Devanshu Asthana, Vaibhav Vishant, Shashwat Sachdev, John Abraham, Garima Mathur, Franz Spilhaus, Kumaar, Vijay Ganguly, Shreya Jain, Daniel Hansen, Jayantilal Gada, Jubin Nautiyal, Dhaval Jayantilal Gada, Aarif Sheikh, Bishwadeep Dipak Chatterjee, Chavi Sirohi, P. S. Vinod, Rohit Chaturvedi, Vishal Mishra, Girish Nakod, Mohd Talib, Mohd Amin Khatib, Vishal Kapoor, Ajay Kapoor, Will Humphris, Ken Metzker, Lakshya Raj Anand, Sumit Batheja
Stream: 
Buy: 
Rent:


# Creating Embeddings and Uploading to Pinecone

In [5]:
index_name = "film-bot-index"

# Create empty index
PINECONE_KEY, PINECONE_INDEX_NAME = os.getenv(
    'PINECONE_API_KEY'), os.getenv('PINECONE_INDEX_NAME')

pc = Pinecone(api_key=PINECONE_KEY)

# Uncomment if index is not created already
# pc.create_index(
#     name="film-bot-index",
#     dimension=1536,
#     metric="cosine",
#     spec=PodSpec(
#         environment="gcp-starter"
#     )
# )

# Target index and check status
pc_index = pc.Index(index_name)
print(pc_index.describe_index_stats())

embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')

vectorstore = PineconeVectorStore(
    pc_index, embeddings
)

# Create record manager
namespace = f"pinecone/{index_name}"
record_manager = SQLRecordManager(
    namespace, db_url="sqlite:///record_manager_cache.sql"
)

record_manager.create_schema()

{'dimension': 1536,
 'index_fullness': 0.00911,
 'namespaces': {'': {'vector_count': 911}},
 'total_vector_count': 911}


In [7]:
def _clear():
    """
    Hacky helper method to clear content.
    """
    index([], record_manager, vectorstore,
          cleanup="full", source_id_key="source")

# Uncomment this line if you want to clear the Pinecone vectorstore
_clear()

index(splits, record_manager, vectorstore,
      cleanup="full", source_id_key="source")

{'num_added': 919, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 0}

In [8]:
query = "Movies based on novels or books."
docs = vectorstore.similarity_search(query)
print(docs[0].page_content)

Keyword: hero, superhero, space travel, based on comic, sequel, teenage girl, aftercreditsstinger, duringcreditsstinger, marvel cinematic universe (mcu), space adventure
Recommendation: The Hunger Games: The Ballad of Songbirds & Snakes, Wonka, Aquaman and the Lost Kingdom, The Killer, Napoleon, Five Nights at Freddy's, The Creator, Wish, Oppenheimer, Killers of the Flower Moon, Migration, Leave the World Behind, A Haunting in Venice, Captain Marvel, Barbie, Rebel Moon - Part One: A Child of Fire, The Equalizer 3, Saltburn, Lift, Elemental, Role Play


# Creating a Retriever

In [9]:
retriever = vectorstore.as_retriever(
    search_type="similarity", search_kwargs={"k": 6})

In [10]:
retrieved_docs = retriever.invoke(
    "What are some films with about a person who becomes a tyrannical president of Panem?")

In [11]:
len(retrieved_docs)

6

In [12]:
print(retrieved_docs[0].page_content)

Title: The Hunger Games: The Ballad of Songbirds & Snakes
Runtime (minutes): 157
Language: en
Overview: 64 years before he becomes the tyrannical president of Panem, Coriolanus Snow sees a chance for a change in fortunes when he mentors Lucy Gray Baird, the female tribute from District 12.
Release Date: 2023-11-15
Genre: Drama, Science Fiction, Action
Keyword: daughter, based on novel or book, arena, fight, mentor, dystopia, riot, exploration, president, prequel, survival, murder, snow, creature, battle, fight to the death, young adult, origin story, based on young adult novel, death game, teenager
Recommendation: Aquaman and the Lost Kingdom, Napoleon, The Marvels, Wonka, The Hunger Games, Rebel Moon - Part One: A Child of Fire, The Family Plan, Oppenheimer, Thanksgiving, Killers of the Flower Moon, Five Nights at Freddy's, Saltburn, The Creator, There's Still Tomorrow, Silent Night, Barbie, Leave the World Behind, Wish, The Hunger Games: Mockingjay - Part 1, Family Switch, Leo


# Creating an LLM with Retriever

In [17]:
from langchain_community.chat_models.huggingface import ChatHuggingFace
from langchain_community.llms import HuggingFaceHub
from huggingface_hub.hf_api import HfFolder 

HfFolder.save_token(os.getenv('HUGGINGFACEHUB_API_TOKEN'))

REPO_ID = "HuggingFaceH4/zephyr-7b-beta"

llm = HuggingFaceHub(
    repo_id=REPO_ID,
    task="text-generation",
    model_kwargs={
        "max_new_tokens": 512,
        "top_k": 30,
        "temperature": 0.1,
        "repetition_penalty": 1.03,
    },
)

# chat_model = ChatHuggingFace(llm=llm)
chat_model = ChatOpenAI(model_name="gpt-3.5-turbo-0125", temperature=0.5)

prompt_template = PromptTemplate.from_template(
    """
    You are an expert on films. Your goal is to recommend films to users based on their
    question and the retrieved context. If you don't know have a recommendation, just say that you don't have one. 
    Format your film recommendations in a bullet point list. Always include the title of the film you are recommending,
    and always provide a justification for why you are recommending a particular film. You should recommend no more than
    five films. If the user asks for film recommendations based on runtime, provide the runtime length in minutes in your
    response. Only recommend films that are given to you in the context, do not recommend any other films.
    
    The documents that are provided will have the following format:

    ### FORMAT ###
    Title: [Title of film]
    Runtime: [Runtime of film] minutes
    Language: [Language of film in ISO 639-1 language code]
    Overview: [Overview of film]
    Release Date: [Release data of film in format YYYY-MM-DD]
    Genre: [Genre of film, a comma separated list]
    Keyword: [Keywords describing the film, a comma separated list]
    Recommendation: [Recommendations for other films based on the current film, a comma separated list]
    Cast: [Cast of film, a comma separated list]
    Crew: [Crew of film, a comma separated list]
    Stream: [Streaming provider to watch film, a comma separated list]
    Buy: [Provider to buy film, a comma separated list]
    Rent: [Provider to rent film, a comma separated list]
    ### FORMAT ###

    Question: {question} 

    Context: {context} 

    Answer:

    """
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | chat_model
    | StrOutputParser()
)

/Users/ed/miniconda3/envs/FilmBot/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_hub.HuggingFaceHub` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


In [37]:
for chunk in rag_chain.stream("Recommend some films similar to Barbie."):
    print(chunk, end="", flush=True)

- Blue Beetle: This film is recommended because it shares the genre of adventure with Barbie and also features a protagonist with special abilities.
- Spider-Man: Across the Spider-Verse: This film is recommended because it shares the genre of adventure with Barbie and also features a superhero protagonist.
- Talk to Me: This film is recommended because it shares the genre of comedy with Barbie and also explores themes of communication and relationships.
- Gran Turismo: This film is recommended because it shares the genre of comedy with Barbie and also features a story centered around cars and racing.
- Indiana Jones and the Dial of Destiny: This film is recommended because it shares the genre of adventure with Barbie and also features a thrilling and action-packed story.

# Creating an Agent with Wiki + RAG

In [1]:
from typing import Optional
from langchain_core.callbacks import CallbackManagerForToolRun
from langchain.agents import AgentExecutor
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_openai_functions_agent, create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import HumanMessagePromptTemplate, MessagesPlaceholder
from langchain_core.tools import BaseTool

# Create tool from retriever
retriever_tool = create_retriever_tool(
    retriever,
    "movie_data",
    """
    Search for basic information regarding films. Information 
    includes: title, release date, original language, genre, 
    recommendations, keywords describing the film, the cast,
    crew, and where to stream, rent, or buy the film. If you
    need any of this information, use this tool!
    """
)

class CustomWikipediaQueryRun(BaseTool):
    """Tool that searches the Wikipedia API."""

    name: str = "wikipedia"
    description: str = (
        """
        A wrapper around Wikipedia. 
        Useful for when you need to answer general questions about 
        people, places, companies, facts, historical events, or other subjects. 
        Input should be a search query.
        """

    )
    api_wrapper: WikipediaAPIWrapper

    def _run(
        self,
        query: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the Wikipedia tool."""
        return self.api_wrapper.run(query)
    

# Create wikipedia tool
wikipedia_tool = CustomWikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
tools = [retriever_tool, wikipedia_tool]

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            'system',
            """
            You are an expert on films. Your goal is to recommend films to users based on their question and the retrieved context. 
            If you don't know have a recommendation, just say that you don't have one. Format your film recommendations in a bullet 
            point list. Always include the title of the film you are recommending, and always provide a justification for why you are 
            recommending a particular film. Your justification should be at least three-four sentences per film. You should recommend no more 
            than five films. If the user asks for film recommendations based on runtime, provide the runtime length in minutes in your response. 
            """
            ),
        MessagesPlaceholder(variable_name='chat_history', optional=True),
        HumanMessagePromptTemplate(prompt=PromptTemplate(
            input_variables=['input'], template='{input}')),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

agent = create_openai_functions_agent(chat_model, tools, prompt_template)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

NameError: name 'retriever' is not defined

In [ ]:
agent_executor.invoke(
    {"input": "Recommend me a list of japanese language films in the science fiction genre made after March 2022."})



> Entering new AgentExecutor chain...

Invoking: `movie_data` with `{'query': 'Japanese science fiction films after March 2022'}`


Title: Warriors of Future
Runtime (minutes): 101
Language: zh
Overview: When a meteor carrying a destructive plant strikes the world, a suicide squad is given hours to save their post-apocalyptic city from total collapse.
Release Date: 2022-08-05
Genre: Drama, Action, Science Fiction
Keyword: alien life-form, meteorite
Recommendation: My Name Is Vendetta, Troll, Emancipation, Guillermo del Toro's Pinocchio, Puss in Boots: The Last Wish, Black Adam, Bullet Train, Shazam! Fury of the Gods, The Woman King, Hidden Figures, Slumberland, Scrooge: A Christmas Carol, The Whale, Oppenheimer, Top Gun: Maverick, Spider-Man: Across the Spider-Verse, Glass Onion: A Knives Out Mystery, Doctor Strange in the Multiverse of Madness, Creed III, Coco, Eternals

Title: Godzilla Minus One
Runtime (minutes): 125
Language: ja
Overview: Postwar Japan is at its lowest point when

{'input': 'Recommend me a list of japanese language films in the science fiction genre made after March 2022.',
 'output': 'Here are some Japanese language science fiction films made after March 2022 that I recommend:\n\n- **Warriors of Future**\n  - **Runtime:** 101 minutes\n  - **Justification:** "Warriors of Future" is a Japanese science fiction film that offers a thrilling post-apocalyptic storyline with a unique twist involving a destructive plant from a meteor. The film combines elements of drama, action, and science fiction, providing an engaging and visually stunning cinematic experience for fans of the genre.\n\n- **Godzilla Minus One**\n  - **Runtime:** 125 minutes\n  - **Justification:** "Godzilla Minus One" is a Japanese science fiction and horror film that delves into postwar Japan\'s struggles with a new crisis in the form of a giant monster. The film explores themes of loss, survival, and the impact of the atomic bomb, making it a compelling watch for those interested in

# Using DataFrame Agent

In [19]:
from langchain_experimental.agents import create_pandas_dataframe_agent
import pandas as pd

# Specify the path to your CSV file
csv_2022 = './data/2022_movie_collection_data.csv'
csv_2023 = './data/2023_movie_collection_data.csv'

# Read the CSV file into a DataFrame
df_2022 = pd.read_csv(csv_2022)
df_2023 = pd.read_csv(csv_2023)

df_all = pd.concat([df_2022, df_2023])

display(df_all)

custom_prefix = """
You are an expert on films. Your goal is to recommend films to users based on their question and the retrieved context. 
If you don't know have a recommendation, just say that you don't have one. Format your film recommendations in a bullet 
point list. Always include the title of the film you are recommending, and always provide a justification for why you are 
recommending a particular film. Your justification should be at least four sentences per film. You should recommend no more 
than five films. If the user asks for film recommendations based on runtime, provide the runtime length in minutes in your response. """
agent = create_pandas_dataframe_agent(
    chat_model, 
    df_all,
    agent_type="openai-tools", 
    verbose=True, 
    prefix=custom_prefix,
    include_df_in_prompt=True, 
    number_of_head_rows=1)

agent.invoke(
    {
        "input": "Recommend films that have a similar relased date to Avatar: The Way of Water?"
    }
)

,Title,Runtime (minutes),Language,Overview,Release Date,Genre,Keyword,Recommendation,Cast,Crew,Stream,Buy,Rent
0,The OctoGames,88,en,"Eight contestants compete in eight deadly, cla...",2022-10-07,"Thriller, Action, Horror","遊戲, 挑戰極限, 組團打怪","Patriots Time, Oppenheimer, The Boy and the He...","Henry Haggard, Sally Harvey Anderson, Claire P...","Galen Christy, Aaron Mirtes, Dylan A. Young, B...",NaN,"Amazon Video, Google Play Movies, YouTube, Vud...","Amazon Video, Google Play Movies, YouTube, Vud..."
1,The Voyagers,108,it,Time jump to 1939 with teenager Max and his fr...,2022-11-21,"Fantasy, Adventure, TV Movie","time travel, zeitreise","Talk to Me, Jurassic World, Il mammone, Enemy ...","Gianmarco Saurino, Federico Tocci, Matteo Schi...","Michele Braga, Matteo Rovere, Gabriele Scarfon...",NaN,NaN,NaN
2,Agent Game,90,en,"Harris, a CIA interrogator at an Agency black ...",2022-04-08,"Action, Thriller","investigation, murder, betrayal, conspiracy, g...","All the Old Knives, Avatar: The Way of Water, ...","Gabby Kono, Adan Canto, Rhys Coiro, Annie Ilon...","Edwin Cude, Jan Rey, Joshua Z. Timmerman, Eric...",Showtime Apple TV Channel,"Apple TV, Amazon Video, Google Play Movies, Yo...","Apple TV, Amazon Video, Google Play Movies, Yo..."
3,Attack,123,hi,"With the Parliament under siege, India’s first...",2022-04-01,"Action, Science Fiction, Thriller","parliament, super soldier, soldier, indian army","Napoleon, Lift, Sixty Minutes, The Marvels, Wi...","Ratna Pathak Shah, Habib Al Aidroos, Prakash R...","Kuldeep Yadav, Akshay Jayantilal Gada, Nakul C...",NaN,NaN,NaN
4,The One Hundred,85,th,A quarantine hotel where overseas travellers s...,2022-10-06,"Thriller, Horror, Science Fiction","quarantine, monster, centipede","Smile, Snag, Barbie, Parasite, Oppenheimer, Mi...","Kamonwan Kanaraksunti, Chanidapa Pongsilpipat,...","Paradorn Vesurai, Kamonwan Kanaraksunti, Soraw...",Amazon Prime Video,"Apple TV, Amazon Video, Google Play Movies, Yo...","Apple TV, Amazon Video, Google Play Movies, Yo..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"Ruby Gillman, Teenage Kraken",91,en,"Ruby Gillman, a sweet and awkward high school ...",2023-06-28,"Animation, Family, Fantasy, Comedy","high school, ocean, boat, transformation, fish...","The Flash, Barbie, Elemental, Transformers: Ri...","Ramona Young, Preston Arsement, Qalil Ismail, ...","Stan Seo, Mike Mitchell, Kirk DeMicco, Robert ...","Netflix, Netflix basic with Ads","Apple TV, Amazon Video, Google Play Movies, Yo...","Apple TV, Amazon Video, Google Play Movies, Yo..."
96,The Wandering Earth II,173,zh,Humans built huge engines on the surface of th...,2023-01-22,"Science Fiction, Action, Drama","space travel, space station, earth in peril, g...","The Wandering Earth, Fast X, Ant-Man and the W...","Tong Liya, Li Xuejian, Gao Ang, Li Ren, Wang L...","Han Cao, Diego Quecano, Ye Xiang, Roc Chen, Fr...",Hoopla,"Apple TV, Amazon Video, Google Play Movies, Yo...","Apple TV, Amazon Video, Google Play Movies, Yo..."
97,Creed III,116,en,"After dominating the boxing world, Adonis Cree...",2023-03-01,"Drama, Action","husband wife relationship, philadelphia, penns...","A Man Called Otto, Avatar: The Way of Water, S...","Natasha Ofili, Al Bernstein, Fernanda Gomez, J...","Kari Barber, Kersten Haff, Alex Leimone, Brenn...","Amazon Prime Video, fuboTV, MGM Plus Amazon Ch...","Apple TV, Amazon Video, Google Play Movies, Yo...","Apple TV, Amazon Video, Google Play Movies, Yo..."
98,Dungeons & Dragons: Honor Among Thieves,134,en,A charming thief and a band of unlikely advent...,2023-03-23,"Adventure, Fantasy, Comedy","platonic love, gang of thieves, dragon, role p...","The Super Mario Bros. Movie, Guardians of the ...","Matthew Waterson, Sophia Lillis, Hugh Grant, J...","Ryō Murakawa, Phil Campbell, Tyler Allison, Da...","Amazon Prime Video, fuboTV, MGM Plus Amazon Ch...","Apple TV, Amazon Video, Google Play Movies, Yo...","Apple TV, Amazon Video, Google Play Movies, Yo..."




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['Release Date'] == '2022-10-07']['Title']"}`


0                      The OctoGames
69    All Quiet on the Western Front
97             Lyle, Lyle, Crocodile
Name: Title, dtype: objectI recommend the following films that have a similar release date to "Avatar: The Way of Water":

- **The OctoGames**: This film could be a good recommendation as it shares the same release date as "Avatar: The Way of Water." It offers a thrilling storyline where eight contestants compete in deadly children's games, making it a captivating watch for fans of intense competition and suspense.

- **All Quiet on the Western Front**: This classic film is known for its powerful depiction of the horrors of war and the human cost of conflict. Released around the same time as "Avatar: The Way of Water," it provides a thought-provoking and emotional viewing experience that resonates with audiences seeking depth and historical

{'input': 'Recommend films that have a similar relased date to Avatar: The Way of Water?',
 'output': 'I recommend the following films that have a similar release date to "Avatar: The Way of Water":\n\n- **The OctoGames**: This film could be a good recommendation as it shares the same release date as "Avatar: The Way of Water." It offers a thrilling storyline where eight contestants compete in deadly children\'s games, making it a captivating watch for fans of intense competition and suspense.\n\n- **All Quiet on the Western Front**: This classic film is known for its powerful depiction of the horrors of war and the human cost of conflict. Released around the same time as "Avatar: The Way of Water," it provides a thought-provoking and emotional viewing experience that resonates with audiences seeking depth and historical context in their movies.\n\n- **Lyle, Lyle, Crocodile**: This film offers a different genre compared to the other recommendations but still carries the charm of a fami

# SQL

In [30]:
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits import create_sql_agent
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

engine = create_engine("sqlite:///movie.db")
# Convert release date to DATE format
df_all['Release Date'] = pd.to_datetime(df_all['Release Date'])

df_all.to_sql("movie", engine, index=False, if_exists='replace')

db = SQLDatabase(engine=engine)

# agent_executor = create_sql_agent(chat_model, db=db, agent_type="openai-tools", verbose=True)

# agent.invoke(
#     {
#         "input": "Recommend films that have a similar relased date to Avatar: The Way of Water?"
#     }
# )

# SQL Toolkit

In [44]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits.sql.prompt import SQL_FUNCTIONS_SUFFIX
from langchain_core.messages import AIMessage, SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)

toolkit = SQLDatabaseToolkit(db=db, llm=ChatOpenAI(temperature=0))
context = toolkit.get_context()
tools = toolkit.get_tools()

print(context)
print(tools)

messages = [
    HumanMessagePromptTemplate.from_template("{input}"),
    AIMessage(content=SQL_FUNCTIONS_SUFFIX),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
]

prompt = ChatPromptTemplate.from_messages(messages)
prompt = prompt.partial(**context)

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

agent = create_openai_tools_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=toolkit.get_tools(),
    verbose=True,
)

{'table_info': '\nCREATE TABLE movie (\n\t"Title" TEXT, \n\t"Runtime (minutes)" BIGINT, \n\t"Language" TEXT, \n\t"Overview" TEXT, \n\t"Release Date" DATETIME, \n\t"Genre" TEXT, \n\t"Keyword" TEXT, \n\t"Recommendation" TEXT, \n\t"Cast" TEXT, \n\t"Crew" TEXT, \n\t"Stream" TEXT, \n\t"Buy" TEXT, \n\t"Rent" TEXT\n)\n\n/*\n3 rows from movie table:\nTitle\tRuntime (minutes)\tLanguage\tOverview\tRelease Date\tGenre\tKeyword\tRecommendation\tCast\tCrew\tStream\tBuy\tRent\nThe OctoGames\t88\ten\tEight contestants compete in eight deadly, classic children\'s games. They seek fame beyond their wil\t2022-10-07 00:00:00\tThriller, Action, Horror\t遊戲, 挑戰極限, 組團打怪\tPatriots Time, Oppenheimer, The Boy and the Heron, Radical, Blue Beetle, Society of the Snow, Role P\tHenry Haggard, Sally Harvey Anderson, Claire Palmer, Adam Pietripaoli, Patrick Sharn, Ali Thomas, Ca\tGalen Christy, Aaron Mirtes, Dylan A. Young, Brad Belemjian\tNone\tAmazon Video, Google Play Movies, YouTube, Vudu, Microsoft Store \tAmazo

In [46]:
agent_executor.invoke(
    {"input": "Recommend me a list of science fiction films in Italian."})



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with ``


movie
Invoking: `sql_db_schema` with `{'table_names': 'movie'}`



CREATE TABLE movie (
	"Title" TEXT, 
	"Runtime (minutes)" BIGINT, 
	"Language" TEXT, 
	"Overview" TEXT, 
	"Release Date" DATETIME, 
	"Genre" TEXT, 
	"Keyword" TEXT, 
	"Recommendation" TEXT, 
	"Cast" TEXT, 
	"Crew" TEXT, 
	"Stream" TEXT, 
	"Buy" TEXT, 
	"Rent" TEXT
)

/*
3 rows from movie table:
Title	Runtime (minutes)	Language	Overview	Release Date	Genre	Keyword	Recommendation	Cast	Crew	Stream	Buy	Rent
The OctoGames	88	en	Eight contestants compete in eight deadly, classic children's games. They seek fame beyond their wil	2022-10-07 00:00:00	Thriller, Action, Horror	遊戲, 挑戰極限, 組團打怪	Patriots Time, Oppenheimer, The Boy and the Heron, Radical, Blue Beetle, Society of the Snow, Role P	Henry Haggard, Sally Harvey Anderson, Claire Palmer, Adam Pietripaoli, Patrick Sharn, Ali Thomas, Ca	Galen Christy, Aaron Mirtes, Dylan A. Young, Brad Belemjian	No

{'input': 'Recommend me a list of science fiction films in Italian.',
 'output': 'I have queried the database for Italian science fiction films, but it seems there are no results matching that criteria. Would you like me to recommend science fiction films in another language or genre?'}